<a href="https://colab.research.google.com/github/abbat1081/neyromanticki/blob/master/6_7_%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#План

1. Импорт библиотек
2. Получить список адресов по котором будем выполнять пасинг
3. Функция парсинга и пополнения списков
4. Сохранение в csv

##Импорт библиотек

In [1]:
import requests #обращение к данным по ссылке
from bs4 import BeautifulSoup # поиск на сайте и парсинг
import numpy as np #для работы с массивами
import pandas as pd # для создания датафрейма

#Получим список URL

In [2]:
url = 'https://habr.com/ru'
firstPage = 352550 # номер  страницы с первым постом
lastPage = 352566 # номер страницы с последним постом

#функция перебора страниц и добавление в список
def get_urls():
  listUrl = [] # пустой список
  for page in range(firstPage, lastPage):
    pageUrl = '{}/post/{}'.format(url, page)
    listUrl.append(pageUrl)
  return listUrl #возвращаем заполненный список


In [3]:
#кол-во ссылок
get_urls()

['https://habr.com/ru/post/352550',
 'https://habr.com/ru/post/352551',
 'https://habr.com/ru/post/352552',
 'https://habr.com/ru/post/352553',
 'https://habr.com/ru/post/352554',
 'https://habr.com/ru/post/352555',
 'https://habr.com/ru/post/352556',
 'https://habr.com/ru/post/352557',
 'https://habr.com/ru/post/352558',
 'https://habr.com/ru/post/352559',
 'https://habr.com/ru/post/352560',
 'https://habr.com/ru/post/352561',
 'https://habr.com/ru/post/352562',
 'https://habr.com/ru/post/352563',
 'https://habr.com/ru/post/352564',
 'https://habr.com/ru/post/352565']

##Функция парсинга и пополнения список

In [4]:
import time

In [6]:
# создаем пустые списки, куда будем закачивать информацию с сайта

titlesList = [] #создаем пустой список под наши заголовки
categoriesList = [] #создаем пустой список под наши категории
postList = [] #создаем пустой список для наших постов
karmaList = [] #пустой списко для кармы
ratingList = [] #пустой список для рейтингов


def get_data():

  for pageData in get_urls():
    fullData =requests.get(pageData) #запрос

    if fullData.status_code == 200: #статус - это успешный отклик страницы

      soup = BeautifulSoup(fullData.text, 'html5lib') # спарсим страницу сайта

      titles = soup.findAll('h1', class_ = 'post__title post__title_full') # парсим заголовки

      for title in range(len(titles)):
        if titles[title].find('span', class_ = 'post__title-text') is not None:
          titlesList.append(titles[title].text.replace('!', ' ')\
                                              .replace('-', '')\
                                              .replace('\n','')\
                                              .replace('.','').lower().strip())
      
      categories = soup.findAll('ul', class_ = 'post__hubs post__hubs_full-post inline-list') #парсим категории

      for category in range(len(categories)):
        if categories[category].find('li', class_ = 'inline-list__item inline-list__item_hub') is not None:
          categoriesList.append(categories[category].text.replace('!', ' ')\
                                              .replace('-', '')\
                                              .replace('\n','')\
                                              .replace('.','').lower().strip())

      posts = soup.findAll('div', class_ = 'post__body post__body_full')

      for post in range(len(posts)):
        if posts[post].find('div', class_ = 'post__text post__text-html post__text_v1') is not None:
          postList.append(posts[post].text.replace('!', ' ')\
                                              .replace('-', '')\
                                              .replace('\n','')\
                                              .replace('.','').lower().strip())
    
      karmas = soup.findAll('a', class_ = 'user-info__stats-item stacked-counter') 

      for karma in range(len(karmas)):
        if karmas[karma].find('div', class_ = 'stacked-counter__value stacked-counter__value_green ') is not None:
          karmaList.append(karmas[karma].text.replace('!', ' ')\
                                              .replace('-', '')\
                                              .replace('\n','')\
                                              .replace('.','').lower().strip())

      ratings = soup.findAll('a', class_ = 'user-info__stats-item stacked-counter stacked-counter_rating')

      for rating in range(len(ratings)):
        if ratings[rating].find('div', class_ = 'stacked-counter__value stacked-counter__value_magenta') is not None:
          ratingList.append(ratings[rating].text.replace('!', ' ')\
                                              .replace('-', '')\
                                              .replace('\n','')\
                                              .replace('.','').lower().strip())
          
    time.sleep(2.0) #пауза для обхода блокировки

  return titlesList, categoriesList, postList, karmaList, ratingList
          




In [12]:
get_data()

([], [], [], [], [])

In [8]:
data_dict = {'title':titlesList, 'categories': categoriesList, 'posts': postList, 'karma': karmaList, 'ratings': ratingList }

In [13]:
datahabr = pd.DataFrame.from_dict(data_dict, orient = 'index')
datahabr = datahabr.transpose() #для того чтобы индексы были сверху
datahabr.head()

,title,categories,posts,karma,ratings


##сохранение в csv

In [14]:
datahabr.to_csv('habr_.csv', sep = ',', index = False) # сохранение в csv